In [1]:
import tensorflow as tf
import keras
from keras import layers
import make_val_prepare_data_aclImdb as imdb_data

max_length = 600
max_tokens = 10000

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length
)

text_only_train_ds = imdb_data.train_ds.map(lambda x,y:x)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = imdb_data.train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = imdb_data.val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = imdb_data.test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)


2024-12-21 00:18:21.485454: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-21 00:18:21.498375: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-21 00:18:21.518103: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-21 00:18:21.525915: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 00:18:21.572926: I tensorflow/core/platform/cpu_feature_guar

Found 22500 files belonging to 2 classes.


I0000 00:00:1734711503.880704   30740 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734711503.940060   30740 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734711503.940131   30740 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734711503.945480   30740 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734711503.952469   30740 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Found 2500 files belonging to 1 classes.
Found 25000 files belonging to 2 classes.
train_ds
input shape: (32,)
input shape: <dtype: 'string'>
targets shape: (32,)
targets shape: <dtype: 'int32'>
input[0]: (32,)
targets[0]: <dtype: 'int32'>
val_ds
input shape: (32,)
input shape: <dtype: 'string'>
targets shape: (32,)
targets shape: <dtype: 'int32'>
input[0]: (32,)
targets[0]: <dtype: 'int32'>
test_ds
input shape: (32,)
input shape: <dtype: 'string'>
targets shape: (32,)
targets shape: <dtype: 'int32'>
input[0]: (32,)
targets[0]: <dtype: 'int32'>


2024-12-21 00:18:47.576567: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [2]:
import transformer_encoder
import position_embedding 

vocab_size = 2000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None, ), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = position_embedding.PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = transformer_encoder.TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer = "rmsprop",
             loss="binary_crossentropy",
             metrics = ["accuracy"])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_embedding            │ (None, None, 256)      │       665,600 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, None, 256)      │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,209,633 (4.61 MB)

 Trainable params: 1,209,633 (4.61 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",save_best_only=True),
    keras.callbacks.TensorBoard(log_dir="full_transformer_encoder")]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20,
         callbacks=callbacks)

Epoch 1/20


I0000 00:00:1734711533.376382   30817 service.cc:146] XLA service 0x7fedf80085b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734711533.376447   30817 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 SUPER, Compute Capability 7.5
2024-12-21 00:18:53.427927: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-21 00:18:53.692946: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90100
I0000 00:00:1734711548.134720   30817 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


704/704 ━━━━━━━━━━━━━━━━━━━━ 292s 386ms/step - accuracy: 0.6221 - loss: 0.7729 - val_accuracy: 0.5920 - val_loss: 0.7676
Epoch 2/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 208s 296ms/step - accuracy: 0.8185 - loss: 0.4058 - val_accuracy: 0.8588 - val_loss: 0.3208
Epoch 3/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 186s 265ms/step - accuracy: 0.8407 - loss: 0.3623 - val_accuracy: 0.6340 - val_loss: 0.7371
Epoch 4/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 154s 220ms/step - accuracy: 0.8547 - loss: 0.3365 - val_accuracy: 0.8728 - val_loss: 0.2973
Epoch 5/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 153s 218ms/step - accuracy: 0.8697 - loss: 0.3095 - val_accuracy: 0.8724 - val_loss: 0.2885
Epoch 6/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 144s 205ms/step - accuracy: 0.8827 - loss: 0.2806 - val_accuracy: 0.7856 - val_loss: 0.4785
Epoch 7/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 140s 199ms/step - accuracy: 0.8939 - loss: 0.2645 - val_accuracy: 0.8820 - val_loss: 0.2699
Epoch 8/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 138s 197ms/step - accuracy: 0.9020 - loss: 0.24

In [5]:
model = keras.models.load_model("full_transformer_encoder.keras",
                                custom_objects={
                                   "full_transformer_encoder": transformer_encoder.TransformerEncoder})

print(f"test acc: {model.evaluate(int_test_ds)[1]:.3f}")

/home/binbin/anaconda3/envs/dl/lib/python3.10/site-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'positional_embedding', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/home/binbin/anaconda3/envs/dl/lib/python3.10/site-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - accuracy: 0.8376 - loss: 0.3613
test acc: 0.838
